In [1]:
# Installing the driver library
pip install cassandra-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 1.8 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthenticator

In [4]:
# Connecting to server and setting the session
cluster = Cluster(["localhost"])
session = cluster.connect()

In [5]:
# Process to Connect to the server on cloud
# from cassandra.cluster import Cluster
# from cassandra.auth import PlainTextAuthProvider

# cloud_config = {
#     'secure_connect_bundle': '/path/to/secure-connect-dbname.zip'
# }
# auth_provider = PlainTextAuthProvider(username='user', password='pass')
# cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
# session = cluster.connect()

In [6]:
# Creating keyspace
query = "create keyspace IF NOT EXISTS keysp with replication={'class':'SimpleStrategy','replication_factor':3};"
session.execute(query)

In [10]:
# getting list of keyspaces
query = "describe keyspaces;"
result = session.execute(query)
for item in result.all():
    print("KeySpace Name: ",item[0])

KeySpace Name:  keysp
KeySpace Name:  system
KeySpace Name:  system_auth
KeySpace Name:  system_distributed
KeySpace Name:  system_schema
KeySpace Name:  system_traces
KeySpace Name:  system_views
KeySpace Name:  system_virtual_schema


In [11]:
# Using the keyspace for creating tables inside
query = "use keysp;"
session.execute(query)

In [12]:
# Creating the table inside the keyspace
query = "create table IF NOT EXISTS emp(id int, name varchar, salary int, dept_id int, primary key(id,dept_id)) with clustering order by (dept_id desc);"
session.execute(query)

In [33]:
# printing the schema of a table
# we have to fetch the schema of table from columns table inside system_schema keypsace
query = 'select column_name,type from system_schema.columns where keyspace_name=%s and table_name=%s;'
session.execute(query,('keysp','emp')).all()

[Row(column_name='dept_id', type='int'),
 Row(column_name='id', type='int'),
 Row(column_name='name', type='text'),
 Row(column_name='salary', type='int')]

In [14]:
# getting list of tables in the keyspace
query = "desc tables;"
tables = session.execute(query).all()
print("tables: ",tables)

tables:  [Row(keyspace_name='keysp', type='table', name='emp')]


In [22]:
# inserting record in table
# In cassandra we have to insert everyting using %s for all type of datatypes
query = 'insert into emp(id,name,salary,dept_id) values(%s,%s,%s,%s);'
session.execute(query,(1,"amit",50000,1))

In [23]:
# adding some more records in table
emp_ids = [2,3,4,5,6]
names = ["ankit","ravi","aman","ram","sam"]
salary = [56000,23342,54640,23450,45670]
dept_ids = [1,2,1,3,2]

query = "insert into emp(id,name,salary,dept_id) values(%s,%s,%s,%s);"
for i in range(len(names)):
    values= ( emp_ids[i], names[i], salary[i], dept_ids[i] )
    session.execute(query,values)

In [28]:
# fetching all the records
query = "select * from emp;"
output = session.execute(query).all()
output

[Row(id=5, dept_id=3, name='ram', salary=23450),
 Row(id=1, dept_id=1, name='amit', salary=50000),
 Row(id=2, dept_id=1, name='ankit', salary=56000),
 Row(id=4, dept_id=1, name='aman', salary=54640),
 Row(id=6, dept_id=2, name='sam', salary=45670),
 Row(id=3, dept_id=2, name='ravi', salary=23342)]

In [34]:
# fetching rows as dictionary
from cassandra.query import dict_factory
session.row_factory = dict_factory
query = "select * from emp;"
output = session.execute(query).all()
output

[{'id': 5, 'dept_id': 3, 'name': 'ram', 'salary': 23450},
 {'id': 1, 'dept_id': 1, 'name': 'amit', 'salary': 50000},
 {'id': 2, 'dept_id': 1, 'name': 'ankit', 'salary': 56000},
 {'id': 4, 'dept_id': 1, 'name': 'aman', 'salary': 54640},
 {'id': 6, 'dept_id': 2, 'name': 'sam', 'salary': 45670},
 {'id': 3, 'dept_id': 2, 'name': 'ravi', 'salary': 23342}]

#### Prepare statement: 
Prepared statements are queries that are parsed by Cassandra and then saved for later use. When the driver uses a prepared statement, 
it only needs to send the values of parameters to bind. This lowers network traffic and CPU utilization within Cassandra because Cassandra 
does not have to re-parse the query each time.

In [39]:
# Using where condition for filtering
# Case 1: when where condition does not contains primary key

# using prepare statement which should be used for better performance
query = session.prepare('select * from emp where id<3 ALLOW FILTERING;')
output = session.execute(query).all()
output

[{'id': 1, 'dept_id': 1, 'name': 'amit', 'salary': 50000},
 {'id': 2, 'dept_id': 1, 'name': 'ankit', 'salary': 56000}]

In [56]:
# Case 2: when where condition contains partition key but value must be = not > or < or >= or <=
query = 'select * from emp where id in (%s,%s,%s);'
output = session.execute(query,[1,2,3]).all()
output

[{'id': 1, 'dept_id': 1, 'name': 'amit', 'salary': 50000},
 {'id': 2, 'dept_id': 1, 'name': 'ankit', 'salary': 56000},
 {'id': 3, 'dept_id': 2, 'name': 'ravi', 'salary': 23342}]

In [58]:
# Case 3: Using partition key and clustering key
query = 'select * from emp where id in (%s,%s,%s) and dept_id=2;'
output = session.execute(query,[1,2,3]).all()
output

[{'id': 3, 'dept_id': 2, 'name': 'ravi', 'salary': 23342}]

In [66]:
# Aggregation operations
session.execute('select sum(salary) as totalSalaryCost from emp;').all()

[{'totalsalarycost': 253102}]

In [59]:
# Creating table with column types list/tuple and dict/map, set, bool, date, string/varchar, text
query = session.prepare('create table mix(id int primary key, name varchar, about text, skills list<varchar>, details map<varchar,varchar>, setval set<text>);')
session.execute(query)

In [62]:
# printint the schema of table
session.execute('select column_name,type from system_schema.columns where keyspace_name=%s and table_name=%s;',('keysp','mix')).all()

[{'column_name': 'about', 'type': 'text'},
 {'column_name': 'details', 'type': 'map<text, text>'},
 {'column_name': 'id', 'type': 'int'},
 {'column_name': 'name', 'type': 'text'},
 {'column_name': 'setval', 'type': 'set<text>'},
 {'column_name': 'skills', 'type': 'list<text>'}]

In [63]:
# Inserting the data
query = session.prepare('insert into mix(id,name,about,skills,details,setval) values(?,?,?,?,?,?);')
values = [
          1,'Amit','I am a DE with experience of 1.5 years in fastpaced startup', 
          ['Hadoop','Hive','NoSQL','Spark'], {'age':'25','gender':'male'}, {'a','b','c','a','d','c'}
         ]
          

session.execute(query,values)
        

In [64]:
# fetching the records
query = 'select * from mix;'
session.execute(query).all()

[{'id': 1,
  'about': 'I am a DE with experience of 1.5 years in fastpaced startup',
  'details': OrderedMapSerializedKey([('age', '25'), ('gender', 'male')]),
  'name': 'Amit',
  'setval': SortedSet(['a', 'b', 'c', 'd']),
  'skills': ['Hadoop', 'Hive', 'NoSQL', 'Spark']}]

In [68]:
# Dropping table
session.execute('drop table emp;')

In [71]:
# Dropping the keyspace
session.execute('drop keyspace keysp;')

In [75]:
output = session.execute('desc keyspaces;').all()
for item in output:
    print("KeySpace: ",item['keyspace_name'])


KeySpace:  system
KeySpace:  system_auth
KeySpace:  system_distributed
KeySpace:  system_schema
KeySpace:  system_traces
KeySpace:  system_views
KeySpace:  system_virtual_schema


In [76]:
# Closing the session
session.shutdown()